In [ ]:
import arviz as az
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import pickle

%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")
from IPython.display import display as print
from core_lib import MyDataset, plot_dataset,r2_loss,get_predict_ks, get_dcdt_func_for_sunode
from core_lib import get_predict_starts
from core_lib import get_model, get_model2, distance_func, MY_EPSILON


In [ ]:
db_csv_path = "dataset/data.csv"
idata_save_path = "runtime/pts/odes-exp04-idata-4-number-1core-c0number-halfnormks-from-core.py-success.dt"

dataset_ori = MyDataset(db_csv_path)
df_ori = dataset_ori.get_df()
cct_names, error_names = dataset_ori.get_var_col_names()
c0 = df_ori[cct_names].iloc[0].values


In [ ]:
# 假设都是一级动力学
k_kinetics = np.repeat(1, 11).astype(np.uint8)
# k_kinetics = np.array([0,0,0,0,1,1,0,0,1,1,0]).astype(np.uint8)
ks = np.array([0.00071942, 0.00269696, 0.00498945, 0.00444931, 0.00571299, 0.00801272, 0.00131931, 0.00319959, 0.00415571, 0.00228432, 0.00177611])
#  =======================================================

# t_eval = np.linspace(0, 150, 100)
t_eval = np.array([0.5, 48, 96, 144])

dataset = MyDataset(db_csv_path)
df = dataset.get_df()
cct_names, error_names = dataset.get_var_col_names()
c0 = df[cct_names].iloc[0].values

dataset.set_as_sim_dataset(t_eval, c0, t0=0.5, args=(ks, k_kinetics))
df = dataset.get_df()

plot_dataset(dataset, dataset)

In [ ]:

mcmc_model = get_model(dataset, t_eval, k_kinetics, distance=distance_func, epsilon=MY_EPSILON, k_sigma_priors=0.01, kf_type=0, c0_type=0)

In [ ]:
# print(get_format_time(),idata_save_path)
# print(ks, c0)
pm.model_to_graphviz(mcmc_model)

In [ ]:
print(idata_save_path)
idata_lv = pickle.load(open(idata_save_path, 'rb'))

# idata_lv = pm.sample_smc(draws=2000, chains=1, model=mcmc_model, progressbar=True)
# pickle.dump(idata_lv,open(idata_save_path, 'wb'))

In [ ]:
# az.plot_trace(idata_lv, kind="rank_vlines");
df[cct_names]

In [ ]:
print(az.summary(idata_lv, round_to=10))
print(ks)

In [ ]:
ks_predict = get_predict_ks(idata_lv)
k_kinetics = np.repeat(1, 11).astype(np.uint8)
t_eval = np.array([0.5, 48, 96, 144])

dataset_new = MyDataset(db_csv_path)
cct_names, error_names = dataset_new.get_var_col_names()
gt_s = dataset_new.get_cct()[0]

dataset_new.set_as_sim_dataset(t_eval, gt_s, t0=0.5, args=(ks_predict, k_kinetics))
plot_dataset(dataset, dataset_new)
# print("r2_loss", r2_loss(dataset_new.get_cct(), dataset.get_cct()))


t_eval_new = np.linspace(0.5, 150, 200)
dataset_new2 = MyDataset(db_csv_path)
dataset_new2.set_as_sim_dataset(t_eval_new, gt_s, t0=0.5, args=(ks_predict, k_kinetics))
plot_dataset(dataset, dataset_new2)